# Calculate LCD data

This notebook computes Local Correlation Differences and subsequent spatial clustering used for figure 6.

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [7]:
p = '/nrs/ahrensraverfish/hesselinkl/RAVERFISH/'

### Load data

In [8]:
from WARP.data_loading import load_WARP_data
from WARP.data_loading import add_subtypes_to_fish_data
from WARP.data_loading import filter_fish_data

fish_inspect = [59, 63, 71]

fish_data = load_WARP_data(data_path=p+'data',
                            filter_genes=['pitx2', 'cx43', 'cfos'], 
                            fish_list=fish_inspect)

fish_data = add_subtypes_to_fish_data(fish_data, ignore_genes_list=['cfos'])

fish_data = filter_fish_data(fish_data, 
                             filter_in_out_of_plane=True, 
                             filter_nan_traces=True, 
                             filter_zero_traces=True)

Loading and preprocessing data for fish F59.


/groups/ahrens/ahrenslab/Luuk/projects/RAVERFISH/WARP/WARP/data_loading.py:389: RuntimeWarning: invalid value encountered in divide
  dff_z = (dff - np.mean(dff, axis=1, keepdims=True))/np.std(dff, axis=1, keepdims=True)
/groups/ahrens/ahrenslab/Luuk/projects/RAVERFISH/WARP/WARP/stimulus_response_utils.py:117: RuntimeWarning: Mean of empty slice
  avg_stim_responses = np.nanmean(stim_responses, axis=2)
/groups/ahrens/home/hesselinkl/anaconda3/envs/warp/lib/python3.10/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Successfully loaded and preprocessed data for fish F59.
Loading and preprocessing data for fish F63.
Successfully loaded and preprocessed data for fish F63.
Loading and preprocessing data for fish F71.
Successfully loaded and preprocessed data for fish F71.
Fish 59: Filtered out 7717 neurons; 134434 neurons remain.
Fish 63: Filtered out 13127 neurons; 153688 neurons remain.
Fish 71: Filtered out 40548 neurons; 160042 neurons remain.


## Code

### Compute averaging masks

#### Compute LCD masks for averages within local neighborhoods

In [5]:
from WARP.lcd_utils import get_LCD_distance_average_masks

LCD_data_dist_avg_masks = get_LCD_distance_average_masks(fish_data, dist_min_list=[0], dist_max_list=[20], min_neurons=5)

/groups/ahrens/home/hesselinkl/anaconda3/envs/warp/lib/python3.10/site-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


#### Compute LCD masks for averages between pairs of brain regions

In [6]:
region_names_inspect_full = [
#     'medulla_oblongata',
    'inferior_medulla_oblongata',
    'intermediate_medulla_oblongata',
    'superior_dorsal_medulla_oblongata', 
    'superior_raphe',
    'cerebellum',
    'tegmentum',
    'locus_coeruleus',
    'nucleus_isthmi', 
    'tectum',
    'periventricular_layer', 
    'tectal_neuropil', 
    'pretectum',
    'prethalamus_(ventral_thalamus)', 
    'dorsal_thalamus_proper',
    'habenula', 
    'hypothalamus',
    'ventral_telencephalon_(subpallium)',
    'dorsal_telencephalon_(pallium)'
]

region_names = fish_data[fish_inspect[0]]['region_data']['region_names']
regions_inspect_inds = np.concatenate([np.where(region_names == (r))[0] for r in region_names_inspect_full]).squeeze()

In [7]:
from WARP.lcd_utils import get_LCD_region_pair_average_masks

LCD_data_region_avg_masks = get_LCD_region_pair_average_masks(fish_data, regions_inspect_inds, region_names[regions_inspect_inds], min_neurons=5)

### Compute LCD data

#### Calculate Local Correlation Difference matrices for all genes

In [8]:
neighbor_radius_inner = 0
neighbor_radius_outer = 15

gene_names = fish_data[fish_inspect[0]]['gene_data']['gene_names']

n_jobs = len(gene_names)
use_dask = True

In [ ]:
if use_dask:

    def load_cluster_kwargs():
        # some dask config settings
        config = {
            'distributed.worker.memory.target': 0.9,
            'distributed.worker.memory.spill': 0.9,
            'distributed.worker.memory.pause': 0.9,
            'distributed.comm.timeouts.connect': '300s',
            'distributed.scheduler.default-task-retries': 1
        }
        
        # set cluster arguments
        cluster_kwargs = {
            'project':'ahrens',
            'ncpus':3,
            'threads':1,
            'min_workers':1,
            'max_workers':100,
            'walltime':'12:00',
            'config':config,
            'cluster_type': 'janelia_lsf_cluster'
        }
    return cluster_kwargs

else:
    def load_cluster_kwargs():
        return None, None

In [ ]:
from WARP.lcd_utils import compute_LCD_all

LCD_data = {}

# ===============================================================================================
# For the sponteneous (random) paradigm, only compute values for distance masks
# For the visual rhapsody (visrap) paradigm, compute values for distance and region masks
# ===============================================================================================

for paradigm in ['random', 'visrap']:
    # Compute LCD matrices, permutation statistics, and optionally mask stats

    if paradigm == 'random':
        mask_dicts={'distance': LCD_data_dist_avg_masks}
        store_mask_values={'distance': True}
    
    if paradigm == 'visrap':
        mask_dicts={'distance': LCD_data_dist_avg_masks, 
                    'region': LCD_data_region_avg_masks}
        store_mask_values={'distance': True,
                           'region': False}

    cluster_kwargs = load_cluster_kwargs()
    
    LCD_data[paradigm] = compute_LCD_all(
        fish_inspect=[59, 63, 71],
        fish_data=fish_data,
        gene_names=gene_names,
        gene_names_inspect=genes_inspect,
        paradigm=paradigm,
        neighbor_radius_inner=neighbor_radius_inner,
        neighbor_radius_outer=neighbor_radius_outer,
        N_permute=2500,
        n_jobs_genes=None,
        p_chunk=50,
        fisher_transform=False,
        store_full_LCD=False,
        mask_dicts=mask_dicts,
        store_mask_values=store_mask_values,
        seed=1,
        cluster_kwargs=cluster_kwargs,
        tmp_root='/groups/ahrens/ahrenslab/Luuk/projects/RAVERFISH/WARP/',
        max_permutation_GB=50
    )

In [ ]:
# Save results
with open('/groups/ahrens/ahrenslab/Luuk/projects/RAVERFISH/WARP/LCD_data.pkl', 'wb') as f:
    pickle.dump(LCD_data, f)

In [3]:
# load results
with open('/groups/ahrens/ahrenslab/Luuk/projects/RAVERFISH/WARP/LCD_data.pkl', 'rb') as f:
    LCD_data = pickle.load(f)

#### Compute LCD clusters from LCD data

In [4]:
from WARP.lcd_clustering_utils import run_multiscale_lcd_clustering_across_genes
from WARP.lcd_statistics import compute_cluster_pvals_all_clusters

# Run clustering across genes
cluster_dict, cluster_dict_by_gene = (
    run_multiscale_lcd_clustering_across_genes(
        LCD_data=LCD_data,
        fish_data=fish_data,
        fish_inspect=fish_inspect,
        stim_key="visrap",
        distance_bin=20,
        target_cluster_size=[150],
        n_neighbors=25,
        min_cluster_size=25,
        alpha=1, # Use alpha of 1 to not exclude any clusters
        n_perm=1,
        permute_within_fish=False,
        min_frac_per_fish=0.5,
        random_state=0,
    )
)


# Comput cluster p-values using the precomputed neuron-wise permutations
cluster_dict = compute_cluster_pvals_all_clusters(
    cluster_dict=cluster_dict,
    LCD_data=LCD_data,
    fish_data=fish_data,
    fish_inspect=fish_inspect,
    stim_key="visrap",
    mask_type="distance",
    mask=20,                  # must match distance_bin
    alternative="two-sided",
    center=None,              # or 'empirical' if you want to center by null mean
)

NameError: name 'fish_data' is not defined

In [ ]:
# Save results
with open('/groups/ahrens/ahrenslab/Luuk/projects/RAVERFISH/WARP/LCD_cluster_dict.pkl', 'wb') as f:
    pickle.dump(cluster_dict, f)

with open('/groups/ahrens/ahrenslab/Luuk/projects/RAVERFISH/WARP/cluster_dict_by_gene.pkl', 'wb') as f:
    pickle.dump(cluster_dict_by_gene, f)